In [35]:
import json
import pickle
import numpy
import pandas
from pandas import DataFrame
from dtw import accelerated_dtw
import math
from math import cos
from math import sin
from math import radians
from math import asin
#pandas.set_option('display.max_rows', 100)

epLat = 0.012160099999998054
epLng = 0.014610749999992125
epTime = 255

In [30]:
with open('trajTrain.pickle', 'rb') as f:
  trajTrainDict = pickle.load(f)
  trajTrain = trajTrainDict['trajTrain']
  trajTrainOri = trajTrainDict['trajTrainOri']
  mappingTrain = trajTrainDict['mappingTrain']



In [ ]:

with open('trajTest.pickle', 'rb') as f:
    trajTrainDict = pickle.load(f)
    trajTest = trajTrainDict['trajTest']
    trajTestOri = trajTrainDict['trajTestOri']
    mappingTest = trajTrainDict['mappingTest']

In [ ]:
with open('trajTest_5.pickle', 'rb') as f:
    trajTrainDict = pickle.load(f)
    trajTest = trajTrainDict['trajTest']
    trajTestOri = trajTrainDict['trajTestOri']
    mappingTest = trajTrainDict['mappingTest']

In [12]:



def getOverlayOneCubeWithPointAmount(trajList1, trajList2, dtime, distX, distY, angDiffConst, pointAmountRatio=0):
    maxTime = trajList1[9] + dtime
    minTime = trajList1[9] - dtime
    trajList3 = trajList2.loc[(trajList2[9] > minTime) & (trajList2[9] < maxTime),]
    if trajList3.size <= 0:
        #print('trajList3.size <= 0')
        return None
    trajList4 = trajList3.loc[abs(trajList3[9] - trajList1[9]) < dtime,]
    if trajList4.size <= 0:
        #print('trajList4.size <= 0, < dtime')
        return None
    trajList4 = trajList4.loc[abs(trajList4[3] - trajList1[3]) < distX,]
    if trajList4.size <= 0:
        #print('trajList4.size <= 0, < distX')
        return None
    trajList4 = trajList4.loc[abs(trajList4[4] - trajList1[4]) < distY,]
    if trajList4.size <= 0:
        #print('trajList4.size <= 0, < distY')
        return None
    trajList4 = trajList4.loc[angleDiff(trajList4[8], trajList1[8]) < angDiffConst,]
    if trajList4.size <= 0:
        #print('trajList4.size <= 0, < angDiffConst')
        return None
    while pointAmountRatio > 0:
      trajList5 = trajList4.loc[abs(trajList4[0] - trajList1[0]) < (trajList1[0] * (1 - pointAmountRatio)),]
      if trajList5.size > 0:
        trajList4 = trajList5
        break
      pointAmountRatio -= 0.1
    if trajList4.size <= 0:
      return None
    n1 = trajList4.shape[0]
    area1 = DataFrame(numpy.zeros((n1, 3), dtype=int))
    area1[0] = trajList4[5].values  # 其它軌
    area1[1] = trajList4[1].values  # 其它軌第幾cube
    area1[2] = trajList4.index.values  # 其它軌此cube在總名單上第幾
    return area1

def getOverlayOneCube(trajList1,trajList2,dtime,distX,distY,angDiffConst):
	maxTime = trajList1[9] + dtime
	minTime = trajList1[9] - dtime
	trajList3 = trajList2.loc[(trajList2[9] > minTime) & (trajList2[9] < maxTime),]
	trajList4 = trajList3.loc[abs(trajList3[9] - trajList1[9]) < dtime,]
	trajList4 = trajList4.loc[abs(trajList4[3] - trajList1[3]) < distX,]
	trajList4 = trajList4.loc[abs(trajList4[4] - trajList1[4]) < distY,]
	trajList4 = trajList4.loc[angleDiff(trajList4[8],trajList1[8]) < angDiffConst,]
	n1 = trajList4.shape[0]
	area1 = DataFrame(numpy.zeros((n1,3), dtype=int))
	area1[0] = trajList4[5].values  #其它軌
	area1[1] = trajList4[1].values  #其它軌第幾cube
	area1[2] = trajList4.index.values  #其它軌此cube在總名單上第幾
	return area1

#設計一個相反也可以的判斷，另外def，但getOverlayOneCube回傳是相反方向要注明，不然算時間找下一點的方向會錯
def angleDiff(a ,b):
	result = abs(a - b)
	result = result.values
	if len(a) > 1:
		index = result > 180
		result[index] = 360 - result[index]
	else:
		if result > 180:
			result = 360 - result
	return result


def getDistance(x,y):
	Long1,Lat1 = x
	Long2,Lat2 = y
	deltaX = cos(radians(Lat2))*cos(radians(Long2)) \
			 - cos(radians(Lat1)) * cos(radians(Long1))
	deltaY = cos(radians(Lat2))*sin(radians(Long2)) \
			 - cos(radians(Lat1)) * sin(radians(Long1))
	deltaZ = sin(radians(Lat2)) - sin(radians(Lat1))
	C = numpy.linalg.norm(numpy.array([deltaX, deltaY, deltaZ]))
	deltaSigma = 2 * asin(C/2)
	d = round( 6371 * 1000 * deltaSigma )
	return d


def getDTWeachRow(row,x,mappingTrain,trajTrainOri):
    tempTrajOriNo = mappingTrain.loc[
            (mappingTrain[2] == row[5]) &
            (mappingTrain[4] == row[1])
            ,3].to_list()
    y = trajTrainOri.loc[
            (trajTrainOri[5] == row[5]) &
            (trajTrainOri[1].isin( tempTrajOriNo ))
            ,[3,4]].to_numpy()
    d, cost_matrix, acc_cost_matrix, path = accelerated_dtw(x,y,getDistance)
    return d


def getTimeByMinDtw(targetNo,
                    area,
                    targetTrajOri,
                    targetMapping,
                    trajTrain,
                    trajTrainOri,
                    mappingTrain):
  trajTrainCubesIndex = area[2]

  targetTrajOriResetIndex = targetTrajOri.set_index(1)
  x = targetTrajOriResetIndex.loc[
    targetMapping.loc[targetMapping[4] == targetNo, 3],
    [3, 4]
  ].to_numpy()

  # result = []
  dfTrajNoWiCubeNo = trajTrain.loc[trajTrainCubesIndex, [5, 1]]
  n = dfTrajNoWiCubeNo.shape[0]
  result = dfTrajNoWiCubeNo.apply(getDTWeachRow, axis=1, args=(x, mappingTrain, trajTrainOri))
  minIndex = result.idxmin()
  minTrajTrain = trajTrain.loc[minIndex,]
  commonTime = trajTrain.loc[minIndex, 10] - trajTrain.loc[minIndex, 9]
  return (commonTime,minIndex,result[minIndex])

# 2020/06/23

def getFullTime2(cube,trajTrainOri,mappingTrain):
    targetTrainOri = trajTrainOri.loc[(trajTrainOri[5] == cube[5]),]
    indexOri = mappingTrain.loc[(mappingTrain[2] == cube[5]) & (mappingTrain[4] == cube[1]) , 3]
    result = targetTrainOri.iloc[indexOri.to_list()[-1],]
    return result['postTime']

def getTestResult4OneTarget2(targetTrajNo,
                             trajTest,
                             trajTestOri,
                             mappingTest,
                             trajTrain,
                             trajTrainOri,
                             mappingTrain,
                             epLng, epLat):
  targetTraj = trajTest.loc[trajTest[5] == targetTrajNo,].copy()
  targetTrajOri = trajTestOri.loc[trajTestOri[5] == targetTrajNo,].copy()
  targetMapping = mappingTest.loc[mappingTest[2] == targetTrajNo,].copy()
  #result = ignoreBeginOrLastOnePointCube(targetTraj, targetTrajOri, targetMapping, epLng, epLat)
  #result = ignoreBeginOrLastOnePointCube_Origi(targetTraj, targetTrajOri, targetMapping)
  #if result is None:
  #  return (0, 0, 0, None)
  #targetNew, targetOriNew = result
  targetNew = targetTraj.copy()
  targetOriNew = targetTrajOri.copy()
  targetNew2 = targetNew.copy()
  realTime = targetNew.iloc[-1, 10] - targetNew.iloc[0, 9]
  realTime2 = realTime
  totalTime = 0
  n = targetNew.shape[0]
  commonTime = 0

  tempExpResult = numpy.zeros((n, 4), dtype='int')
  tempExpResult[:, 0] = targetNew.iloc[:, 10] - targetNew.iloc[:, 9]
  listArea = []
  nextPointTime = 0
  prePointTime = 0
  preCubeTimeDiff = 0
  preNextPointTime = 0
  countAreaNone = 0
  for i2 in range(n):
    if i2 > 0:
      targetNew.iloc[i2, 9] = targetNew.iloc[i2 - 1, 9] + commonTime
    tau = 3600
    maxAngle = 10
    area = getOverlayOneCubeWithPointAmount(targetNew.iloc[i2,],
                                            trajTrain,
                                            tau,
                                            epLng,
                                            epLat,
                                            maxAngle, 0.5)
    if area is None:
      countAreaNone += 1
      print('ERROR: index=', i2, ' area is None')
      flagGiveUp = False
      epLngEpLatMultiply = 1
      while area is None:
            if tau < 86400:
                tau += 3600
            elif maxAngle < 180:
                maxAngle = 180
            elif epLngEpLatMultiply < 10:
                epLngEpLatMultiply += 1
            else:
                flagGiveUp = True
                break
            area = getOverlayOneCubeWithPointAmount(targetNew.iloc[i2,],
                                            trajTrain,
                                            tau,
                                            epLng*epLngEpLatMultiply,
                                            epLat*epLngEpLatMultiply,
                                            maxAngle, 0.5)
      if flagGiveUp:
        print('Give Up!')
        continue
    
    commonTime,minIndex,dtwResult = getTimeByMinDtw(targetNew.iloc[i2, 1],
                                 area,
                                 targetOriNew,
                                 targetMapping,
                                 trajTrain,
                                 trajTrainOri,
                                 mappingTrain)
    commonTime2 = getFullTime2(trajTrain.loc[minIndex,],trajTrainOri,mappingTrain)
    commonTime += commonTime2
    tempExpResult[i2, 2] = commonTime
    totalTime += commonTime
    tempExpResult[i2, 3] = dtwResult
    confidence = float(n - countAreaNone) /float(n)
  return (totalTime, realTime, realTime2, tempExpResult, confidence)



In [13]:
targetTrajNo = 424
totalTime,realTime,realTime2,tempExpResult,confidence = \
    getTestResult4OneTarget2(targetTrajNo,
                            trajTest,
                            trajTestOri,
                            mappingTest,
                            trajTrain,
                            trajTrainOri,
                            mappingTrain,
                            epLng,epLat)

ERROR: index= 0  area is None
ERROR: index= 1  area is None
ERROR: index= 2  area is None


In [14]:
print(totalTime,"\n",realTime,"\n",realTime2,"\n",tempExpResult,"\n",confidence)

982.0 
 1261 
 1261 
 [[ 250    0  199 4290]
 [  21    0  271 2499]
 [ 231    0  139 1249]
 [  59    0   90  517]
 [  90    0  122  896]
 [ 121    0   80  646]
 [  69    0   81  522]] 
 0.5714285714285714


In [4]:
trajNoList = numpy.unique(trajTest[5])

expResult = numpy.zeros((trajNoList.max()+1,6),dtype='int')
#expResult[:,2] = 0
expResult[:,4] = list(range(trajNoList.max()+1))
#
#expTotalResult = 0

In [ ]:
#
for targetTrajNo in trajNoList[:800]:
    print(targetTrajNo, end=' ')
    totalTime,realTime,realTime2,tempExpResult,confidence = \
        getTestResult4OneTarget2(targetTrajNo,
                            trajTest,
                            trajTestOri,
                            mappingTest,
                            trajTrain,
                            trajTrainOri,
                            mappingTrain,
                            epLng,epLat)
    expResult[targetTrajNo,0] = 1
    expResult[targetTrajNo,1] = totalTime
    expResult[targetTrajNo,3] = realTime
    expResult[targetTrajNo,5] = confidence

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 ERROR: index= 4  area is None
163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 27

In [19]:
expResult[800:1401]

array([[   0,    0,    0,    0,  800,    0],
       [   1, 2182,    0, 2171,  801,    1],
       [   1, 2121,    0, 1809,  802,    1],
       ...,
       [   1, 1474,    0, 1057, 1398,    1],
       [   1, 2171,    0, 1320, 1399,    1],
       [   1,  628,    0, 1065, 1400,    1]])

In [36]:
with open('expResult_trajTest_5.pickle', 'rb') as f:
      expResult_5 = pickle.load(f)

In [38]:
with open('expResult_trajTest_5_10.pickle', 'rb') as f:
      expResult_5_10 = pickle.load(f)

In [39]:
with open('expResult_trajTest_10_15.pickle', 'rb') as f:
      expResult_10_15 = pickle.load(f)

In [40]:
with open('expResult_trajTest_15_20.pickle', 'rb') as f:
      expResult_15_20 = pickle.load(f)

In [41]:
with open('expResult_trajTest_20_25.pickle', 'rb') as f:
      expResult_20_25 = pickle.load(f)

In [42]:
with open('expResult_trajTest_25_30.pickle', 'rb') as f:
      expResult_25_30 = pickle.load(f)

In [47]:
expResult = numpy.concatenate( (expResult_5[1:], expResult_5_10[1:], expResult_10_15[1:], expResult_15_20[1:], expResult_20_25[1:], expResult_25_30[1:]) )

In [23]:
with open('expResult1-1401.pickle', 'wb') as f:
      pickle.dump(expResult,f)

In [48]:
expResult

array([[1.00000000e+00, 6.04000000e+02, 0.00000000e+00, 3.78000000e+02,
        1.00000000e+00, 1.00000000e+00],
       [1.00000000e+00, 2.18300000e+03, 0.00000000e+00, 2.08200000e+03,
        2.00000000e+00, 1.00000000e+00],
       [1.00000000e+00, 9.40000000e+02, 0.00000000e+00, 1.06600000e+03,
        3.00000000e+00, 1.00000000e+00],
       ...,
       [1.00000000e+00, 1.54100000e+03, 0.00000000e+00, 1.42900000e+03,
        4.00000000e+00, 9.33333333e-01],
       [1.00000000e+00, 1.75100000e+03, 0.00000000e+00, 1.98900000e+03,
        5.00000000e+00, 1.00000000e+00],
       [1.00000000e+00, 2.07800000e+03, 0.00000000e+00, 2.43200000e+03,
        6.00000000e+00, 1.00000000e+00]])

In [ ]:
expResult[1:100]

In [27]:
expResult[1:801,:5] = expResult_1_800[1:801]

In [52]:
expResult[:,5]

array([1.        , 1.        , 1.        , ..., 0.93333333, 1.        ,
       1.        ])

In [50]:
#expResult2 = expResult[numpy.where((expResult[:,0] == 1) & (expResult[:,1] >= 1000) )]
#expResult2 = expResult[1:1401]
expResult2 = expResult[numpy.where((expResult[:,5] == 1) )]

deviation = numpy.absolute(expResult2[:,1] - expResult2[:,3])
eachAPE = deviation * 100 / expResult2[:,3] 
#eachAPE = eachAPE.astype(int)
#eachAPE[[49,6,11,60]] = 0
eachAPE

array([59.78835979,  4.85110471, 11.81988743, ..., 13.36283186,
       11.96581197, 14.55592105])

20.52642857142857

269.7257142857143

In [51]:
#MAPE
print('MAPE=', eachAPE.mean())
#RMSE
print('RMSE=', numpy.sqrt( (numpy.square(deviation) ).mean() ) )
#MAE
print('MAE=', deviation.mean() )

MAPE= 20.899779733013784
RMSE= 416.64437059031235
MAE= 267.60913705583755
